1. 문서의 내용을 읽는다
2. 문서를 쪼갠다
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래걸림   
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [1]:
#%pip install -qU  docx2txt 

In [2]:
# %pip install -qU langchain-text-splitters

In [3]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 200 # 겹치는 단위
)

loader = Docx2txtLoader('./tax.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [4]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model = 'text-embedding-3-large')

In [5]:
from langchain_chroma import Chroma

# database = Chroma.from_documents(documents = document_list, embedding = embedding, persist_directory = "./chroma", collection_name='chroma-tax')
database = Chroma(
    embedding_function=embedding, persist_directory="./chroma", collection_name="chroma-tax"
)

In [6]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query)

In [7]:
retrieved_docs

[Document(id='395badf6-8024-416b-bc3a-0d837437bc1d', metadata={'source': './tax.docx'}, page_content='[전문개정 2009. 12. 31.]\n\n\n\n제10조(납세지의 변경신고) 거주자나 비거주자는 제6조부터 제9조까지의 규정에 따른 납세지가 변경된 경우 변경된 날부터 15일 이내에 대통령령으로 정하는 바에 따라 그 변경 후의 납세지 관할 세무서장에게 신고하여야 한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제11조(과세 관할) 소득세는 제6조부터 제10조까지의 규정에 따른 납세지를 관할하는 세무서장 또는 지방국세청장이 과세한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2장 거주자의 종합소득 및 퇴직소득에 대한 납세의무 <개정 2009. 12. 31.>\n\n\n\n제1절 비과세 <개정 2009. 12. 31.>\n\n\n\n제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 20., 2018. 3. 20., 2018. 12. 31., 2019. 12. 10., 2019. 12. 31., 2020. 6. 9., 2020. 12. 29., 2022. 8. 12., 2022. 12. 31., 2023. 8. 8., 2023. 12. 31., 2024. 12. 31., 2025. 10. 1., 2025. 12. 23.>\n\n1. 「공익신탁법」에 따른 공익신탁의 이익\n\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n\n나. 1개의 주택을 소유하는 자의 주택임대소득(제99조

In [8]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = 'gpt-4o')


In [9]:
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다
- [Context]를 참고해서 사용자의 질문에 답변해주세요

[Context]
{retrieved_docs}

Question: {query}
"""

In [10]:
ai_message = llm.invoke(prompt)

In [11]:
ai_message.content

'연봉 5천만 원인 직장인의 소득세를 계산하기 위해서는 몇 가지 추가 정보가 필요합니다. 예를 들어, 해당 직장인의 기본공제, 특별공제, 연금보험료, 건강보험료, 고용보험료 등 공제 항목들을 고려해야 합니다. 그러나 기본적인 소득세율을 적용하여 대략적인 소득세를 계산할 수 있습니다.\n\n2023년 기준으로 한국 소득세율은 다음과 같습니다:\n- 1,200만 원 이하: 6%\n- 1,200만 원 초과 ~ 4,600만 원 이하: 15%\n- 4,600만 원 초과 ~ 8,800만 원 이하: 24%\n- 8,800만 원 초과 ~ 1억 5천만 원 이하: 35%\n- 1억 5천만 원 초과 ~ 3억 원 이하: 38%\n- 3억 원 초과 ~ 5억 원 이하: 40%\n- 5억 원 초과: 42%\n\n예를 들어, 연봉 5천만 원의 경우:\n\n1. 1,200만 원 * 6% = 72만 원\n2. (4,600만 원 - 1,200만 원) * 15% = 510만 원\n3. (5,000만 원 - 4,600만 원) * 24% = 96만 원\n\n총 소득세: 72만 원 + 510만 원 + 96만 원 = 678만 원\n\n위 계산은 공제를 반영하지 않은 매우 간략한 예시입니다. 실제로는 각종 공제를 적용하여 최종 소득세가 결정되므로 구체적인 계산은 개인의 세부 정보에 따라 달라질 수 있습니다. 정확한 금액을 확인하기 위해서는 국세청 홈택스를 통해 소득공제 항목 등을 자세히 확인 후 계산하는 것이 필요합니다.'

In [12]:
# %pip install -U langchain langchainhub --quiet

In [13]:
from langchain_classic import hub

prompt = hub.pull("rlm/rag-prompt")

In [14]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [15]:
from langchain_classic.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs = {"prompt":prompt}
)

In [16]:
ai_message = qa_chain({"query":query})

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_19176\980961933.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  ai_message = qa_chain({"query":query})


In [17]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '저는 소득세 계산에 필요한 정보를 가지고 있지 않습니다. 연봉 5천만 원의 소득세를 계산하려면 현재 한국의 소득세율과 공제를 고려해야 하며, 이를 위해서는 세무 관련 자료나 전문가의 도움이 필요합니다.'}